## Реализации различных хранилищ с асинхронностью

Известно что компоненты должны зависеть не от реализаций, а от интерфейсов.

Имеется некоторое приложение app.py, которое будет осуществлять работу с аккаунтами. Основной класс этого приложения `AccountManager` зависит от храналища аккаунтов, интерфейс которого предоставлен `AccountsStorageProtocol`. Необходимо написать 3 реализации хранилища, соответсвенно `AccountsPostgresStorage`, `AccountsMongoStorage` и `AccountsRedisStorage`. Для того чтобы убедиться в том что реализации работают правильно, достаточно запустить app.py: сработает тестовая функция `test_main`

У приложения есть также тестовая реализация test_app.py и она уже работает, правда использует вместо настоящего персистентного хранилища заглушку MockAccountsStorage. Из MockAccountsStorage можно стащить часть логики

Можете добавлять в ваши реализации любые методы, помните что для соответствия интерфейсу достаточно соблюдать только сигнатуры из протокола


Для ознакомления с API Баз данных, смотри документацию (и материал лекции):

    https://www.psycopg.org/docs/
    https://redis.readthedocs.io/en/latest/
    https://pymongo.readthedocs.io/en/stable/
    
Сами базы данных можно установить локально при помощи Docker
- https://hub.docker.com/_/postgres
- https://hub.docker.com/_/mongo
- https://hub.docker.com/_/redis

Также требуется реализовать асинхронный вариан для каждой из реализаций и переписать `AccountManager` и `test_main` так, чтобы они использовали преимущества асинхронной работы

## Распределённая блокировка

Стандартная ситуация, когда над одной задачей работают несколько воркеров, и в процессе обработки, эти воркеры должны обращаться к общему ресурсу. Допустим, 1млн хешей обсчитываются 10-ю физическими машинами и скидывают результаты своей работы в 1 файл. Что будет если 2 машины одновременно откроют файл на добавление и положат в него новые данные? Правильно - одна машина перезапишет результата работы второй машины ... или наоборот. Для того чтобы такого не происходило, используется механизм распределённой блокировки: машина, которая будет готова положить результат своей работы, сначала попробует установить блокировку
- в случае успеха, машина сделает необходимую работу и отпустит блокировку
- в случае если блокировка уже кем-то поставлена, машина будет ожидать
Распределённая блокировка может быть реализована с использованием любого внешнего хранилища, обычно применяется redis

Требуется реализовать распределённую блокировку на 
- redis
- mongoDB
- postgreSQL
такую, чтобы с любой из реализаций, работал следующий код:

In [ ]:
import time
from concurrent.futures import ThreadPoolExecutor

mu = Lock()  # вот его нужно реализовать
result = 0

def function():
     with mu:
        global result
        r = result
        time.sleep(1)
        result = r + 1

    
def main():  
    with ThreadPoolExecutor(max_workers=5) as executor:
        for _ in range(10):
            executor.submit(function)
    print(result)  # хотим получить в итоге 10
    
main()

## Асинхронное взаимодействие

Есть модуль `hident.py`, который содержит логику обработки криптографических хешей. Функция `identify_hashes(input_hash)` позволяет узнать алгоритм хеширования по виду самого хеша, корутина `long_solve_hash(input_hash, alg)` имитирует работу 'обратного преобразования' хеша ([как это?](https://hashcat.net/hashcat/)), то есть позволяет восстановить секретное слово
 
 Требуется на [aiohttp](https://docs.aiohttp.org/en/stable/web.html) разработать приложение (сервер), которое будет иметь 3 ручки:
 - для вычисления возможного алгоритма хеширования по хешу `host:port/define/{hash}`
 - 2 ручки для асинхронного вычисления секретного слова (нужен именно асинхронный вариант, так как long_solve_hash работает очень долго - представьте надоедливое вращающееся колёсико загрузки, которое было бы в синхронном варианте)
     - `host:port/createSolveTask?hash={hash}&algorithm={algorithm}`
     - `host:port/getPassword?taskId={tid}`
 
 Напишите клиента, который будет использовать сервер:
 - отправлять 1 любой хеш, например `c4ca4238a0b923820dcc509a6f75849b`, получать его возможные алгоритмы 
 - по каждому алгоритму вычислять секретное слово
 
 Полностью контракт сервиса может выглядеть так:
 ```go
rpc createSolveTask (CreateSolveTaskIn) CreateSolveTaskOut `Создать задачу на преобразование хеша`

message CreateSolveTaskIn  {inputHash string, alg string}
message CreateSolveTaskOut {taskId int}

rpc getPassword (GetPasswordIn) GetPasswordOut `Получить результат преобразования`

message GetPasswordIn      {taskId int}
message GetPasswordOut     {password *string, err *Error}

rpc define (DefineIn) DefineOut `Определить алгоритм`

message DefineIn      {hash string}
message DefineOut     {algs []string}

message Error              {code int, message string}  `допустимые значения в ErrCode, ErrMessage`
const ErrCode {
    ERR_TASK_NOT_FOUND          = 1000
    ERR_TASK_NOT_FINISHED       = 1001
}
const ErrMessage {
    ERR_TASK_NOT_FOUND          = "task not found"
    ERR_TASK_NOT_FINISHED       = "task not finished"
}
```